In [91]:
import pandas as pd
import numpy as np

In [210]:
province = pd.read_pickle('.\Data\province.pkl')

In [211]:
def get_measures(level, stats,prov, df):
        level_dict = {}
        stats_dict = {}
        for ele in level:
            for sta in stats:
                stats_dict[sta] = list(df[(df.Statistics == sta) & (df.Levelofstudy == ele) & (df.Province_of_Study == prov)].Value)
            level_dict[ele] = dict(stats_dict)
        return level_dict

In [209]:
p_level = np.unique(province.Levelofstudy)
p_stats = np.unique(province.Statistics)
p_prov = np.unique(province.Province_of_Study)

In [169]:
def get_measures_province(p_prov):
    province_measure_dict = {}
    for prov in p_prov:
        province_measure_dict[prov] = get_measures(p_level,p_stats,prov,province)
    return province_measure_dict

In [170]:
prov_measures = get_measures_province(p_prov)

In [172]:
def percent_inc_dec(y,x):
        return ((y-x)/x)*100 if x != 0 else 0

In [189]:
def cal_percent(prov_measures):
    prov_measures_percent = {}
    level_dict = {}
    stats_dict = {}
    for prov_key in prov_measures.keys():
        for level_key in prov_measures[prov_key].keys():
            for stat_key in prov_measures[prov_key][level_key].keys():
                tmp_list = [0]
                for i in range(len(prov_measures[prov_key][level_key][stat_key])-1):
                    tmp_list.append(percent_inc_dec(prov_measures[prov_key][level_key][stat_key][i+1],
                                                    prov_measures[prov_key][level_key][stat_key][i]))
                stats_dict[stat_key] = tmp_list
            level_dict[level_key] = dict(stats_dict)
        prov_measures_percent[prov_key] = dict(level_dict)

    return prov_measures_percent

In [190]:
prov_measures_percent = cal_percent(prov_measures)

In [191]:
province['Percent'] = pd.Series(dtype='float64')
province

,Year,Province_of_Study,Dguid,Levelofstudy,Statistics,Unit_of_measure,Value,Percent
0,2000,Newfoundland and Labrador,2016A000210,College,Number of graduates,Number,1700.0,NaN
1,2000,Newfoundland and Labrador,2016A000210,College,Females,Percent,50.0,NaN
2,2000,Newfoundland and Labrador,2016A000210,College,Average age at graduation,Years,26.0,NaN
3,2000,Newfoundland and Labrador,2016A000210,College,Median age at graduation,Years,23.0,NaN
4,2000,Newfoundland and Labrador,2016A000210,College,Under age 25 at graduation,Percent,63.0,NaN
...,...,...,...,...,...,...,...,...
955,2015,British Columbia,2016A000259,Doctorate,Females,Percent,49.0,NaN
956,2015,British Columbia,2016A000259,Doctorate,Average age at graduation,Years,35.0,NaN
957,2015,British Columbia,2016A000259,Doctorate,Median age at graduation,Years,32.0,NaN
958,2015,British Columbia,2016A000259,Doctorate,Under age 25 at graduation,Percent,0.0,NaN


In [193]:
def get_percents(prov_measures_percent,df):
    for prov_key in prov_measures_percent.keys():
        for level_key in prov_measures_percent[prov_key].keys():
            for stat_key in prov_measures_percent[prov_key][level_key].keys():
                index_list = list(df[(df.Statistics == stat_key) & (df.Levelofstudy == level_key) & (df.Province_of_Study == prov_key)].index)
                for val_index,val in enumerate(index_list):
                    value = prov_measures_percent[prov_key][level_key][stat_key][val_index]
                    df.loc[val,'Percent'] = value
    return df

In [195]:
province = get_percents(prov_measures_percent,province)

In [197]:
province[(province.Province_of_Study == "Alberta") & 
         (province.Levelofstudy == "Doctorate") & 
         (province.Statistics == "Females")]

,Year,Province_of_Study,Dguid,Levelofstudy,Statistics,Unit_of_measure,Value,Percent
211,2000,Alberta,2016A000248,Doctorate,Females,Percent,43.0,0.000000
451,2005,Alberta,2016A000248,Doctorate,Females,Percent,48.0,11.627907
691,2010,Alberta,2016A000248,Doctorate,Females,Percent,46.0,-4.166667
931,2015,Alberta,2016A000248,Doctorate,Females,Percent,46.0,0.000000


In [208]:
province.to_csv('.\Data\province_transformed.csv')